In [ ]:
#agregacja w oknach czasowych
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def process_batch(df, batch_id):
    print(f"Batch ID: {batch_id}")
    df.show(truncate=False)

spark = SparkSession.builder.appName("RealTimeEcommerce").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

schema = StructType([
    StructField("user_id", StringType()),
    StructField("event_type", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("product_id", StringType()),
    StructField("category", StringType()),
    StructField("price", DoubleType())
]) 
stream = (spark.readStream
          .schema(schema)
          .json("data/stream"))
#thumbling window
windowed = (stream.withWatermark("timestamp", "1 minute")
           .groupBy(window("timestamp", "5 minutes"), "event_type")
           .count()
)
 
# pamietaj, że agregacje wymagają opcji complete
query = (windowed
         .writeStream
         .outputMode("update")
         .format("console")
         .foreachBatch(process_batch)
         .start()
        )
